In [1]:
import sys
sys.path.append("../") # adding the path to read the utils file
import pandas as pd
import glob
from utils import *

pd.set_option('display.max_columns', None)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Preprocessing the data

In [2]:
# Define the pattern to match the file names
file_pattern = '../datasets/CICIDS-2018/*.csv'

# Retrieve the list of file paths matching the pattern
file_paths = glob.glob(file_pattern)

def load_ds(path):
    print(path)
    return pd.read_csv(path, encoding="us-ascii", encoding_errors="ignore", usecols=cicids_18_col, dtype_backend="pyarrow")

ds = pd.concat([load_ds(file_path) for file_path in file_paths], ignore_index=True)
ds.columns = ['Protocol', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
       'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min', 'Label']
ds.head()

../datasets/CICIDS-2018/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
../datasets/CICIDS-2018/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv


,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,37953,5,3,135.0,127.0,135.0,0.0,27.0,60.373835,127.0,0.0,42.333333,73.323484,6903.275103,210.787026,5.421857e+03,5.403580e+03,12099.0,23.0,37953.0,9488.25,3.245485e+03,12382.0,6013.0,19960.0,9980.0,13546.75171,19559.0,401.0,0,0,0,0,168,104,131.741891,79.045135,0.0,135.0,29.111111,57.800183,3340.861111,0,0,0,1,0,0,0,0,0.0,32.750000,27.0,42.333333,0,0,0,0,0,0,5,135,3,127,29200,219,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,17,117573474,3,0,1500.0,0.0,500.0,500.0,500.0,0.000000,0.0,0.0,0.000000,0.000000,12.757980,0.025516,5.880000e+07,2.380000e+07,75600000.0,42000000.0,118000000.0,58800000.00,2.380000e+07,75600000.0,42000000.0,0.0,0.0,0.00000,0.0,0.0,0,0,0,0,24,0,0.025516,0.000000,500.0,500.0,500.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,666.666667,500.0,0.000000,0,0,0,0,0,0,3,1500,0,0,-1,-1,2,8,0.0,0.0,0.0,0.0,58800000.0,23800000.0,75600000.0,42000000.0,Benign
2,17,117573474,3,0,1500.0,0.0,500.0,500.0,500.0,0.000000,0.0,0.0,0.000000,0.000000,12.757980,0.025516,5.880000e+07,2.380000e+07,75600000.0,42000000.0,118000000.0,58800000.00,2.380000e+07,75600000.0,42000000.0,0.0,0.0,0.00000,0.0,0.0,0,0,0,0,24,0,0.025516,0.000000,500.0,500.0,500.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,666.666667,500.0,0.000000,0,0,0,0,0,0,3,1500,0,0,-1,-1,2,8,0.0,0.0,0.0,0.0,58800000.0,23800000.0,75600000.0,42000000.0,Benign
3,17,99743998,5,0,2500.0,0.0,500.0,500.0,500.0,0.000000,0.0,0.0,0.000000,0.000000,25.064165,0.050128,2.490000e+07,3.400000e+07,75600000.0,4000290.0,99700000.0,24900000.00,3.400000e+07,75600000.0,4000290.0,0.0,0.0,0.00000,0.0,0.0,0,0,0,0,40,0,0.050128,0.000000,500.0,500.0,500.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,600.000000,500.0,0.000000,0,0,0,0,0,0,5,2500,0,0,-1,-1,4,8,4000290.0,0.0,4000290.0,4000290.0,31900000.0,37900000.0,75600000.0,7200397.0,Benign
4,17,99743999,5,0,2500.0,0.0,500.0,500.0,500.0,0.000000,0.0,0.0,0.000000,0.000000,25.064165,0.050128,2.490000e+07,3.400000e+07,75600000.0,4000286.0,99700000.0,24900000.00,3.400000e+07,75600000.0,4000286.0,0.0,0.0,0.00000,0.0,0.0,0,0,0,0,40,0,0.050128,0.000000,500.0,500.0,500.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,600.000000,500.0,0.000000,0,0,0,0,0,0,5,2500,0,0,-1,-1,4,8,4000286.0,0.0,4000286.0,4000286.0,31900000.0,37900000.0,75600000.0,7200399.0,Benign


In [3]:
compute_ratio(ds,'Label')

                          Ratio
Label                          
Benign                    83.07
DDOS attack-HOIC           4.23
DDoS attacks-LOIC-HTTP     3.55
DoS attacks-Hulk           2.85
Bot                        1.76
FTP-BruteForce             1.19
SSH-Bruteforce             1.16
Infilteration              1.00
DoS attacks-SlowHTTPTest   0.86
DoS attacks-GoldenEye      0.26
DoS attacks-Slowloris      0.07
DDOS attack-LOIC-UDP       0.01
Brute Force -Web           0.00
Brute Force -XSS           0.00
SQL Injection              0.00


In [4]:
original_lenght = ds.shape[0]

In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16232988 entries, 0 to 16232987
Data columns (total 78 columns):
 #   Column                       Dtype          
---  ------                       -----          
 0   Protocol                     int64[pyarrow] 
 1   Flow Duration                int64[pyarrow] 
 2   Total Fwd Packets            int64[pyarrow] 
 3   Total Backward Packets       int64[pyarrow] 
 4   Total Length of Fwd Packets  double[pyarrow]
 5   Total Length of Bwd Packets  double[pyarrow]
 6   Fwd Packet Length Max        double[pyarrow]
 7   Fwd Packet Length Min        double[pyarrow]
 8   Fwd Packet Length Mean       double[pyarrow]
 9   Fwd Packet Length Std        double[pyarrow]
 10  Bwd Packet Length Max        double[pyarrow]
 11  Bwd Packet Length Min        double[pyarrow]
 12  Bwd Packet Length Mean       double[pyarrow]
 13  Bwd Packet Length Std        double[pyarrow]
 14  Flow Bytes/s                 double[pyarrow]
 15  Flow Packets/s               d

In [6]:
def downcast_df(df):
    # Identify integer columns for downcast
    int_cols = df.select_dtypes(include='int64').columns

    # Downcast integer columns to unsigned integers
    for col in int_cols:
        df[col] = pd.to_numeric(df[col], downcast='unsigned')

    # Identify float columns for downcast
    float_cols = df.select_dtypes(include='float64').columns

    # Downcast float columns to smaller floating-point types
    for col in float_cols:
        df[col] = pd.to_numeric(df[col], downcast='float')

    return df

In [7]:
ds = downcast_df(ds)

In [8]:
ds.dropna(inplace=True)

In [9]:
droped_na_lenght = ds.shape[0]

In [10]:
print("removed", original_lenght - droped_na_lenght, "null values")

removed 59766 null values


In [11]:
ds.drop_duplicates(inplace=True)

In [12]:
print("removed", droped_na_lenght - ds.shape[0], "duplicates")

removed 5351241 duplicates


removing the unecessary columns, like port, ip, timestap and flow id

In [13]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10821981 entries, 0 to 16232987
Data columns (total 78 columns):
 #   Column                       Dtype          
---  ------                       -----          
 0   Protocol                     uint8[pyarrow] 
 1   Flow Duration                int64[pyarrow] 
 2   Total Fwd Packets            uint32[pyarrow]
 3   Total Backward Packets       uint32[pyarrow]
 4   Total Length of Fwd Packets  double[pyarrow]
 5   Total Length of Bwd Packets  double[pyarrow]
 6   Fwd Packet Length Max        float[pyarrow] 
 7   Fwd Packet Length Min        float[pyarrow] 
 8   Fwd Packet Length Mean       double[pyarrow]
 9   Fwd Packet Length Std        double[pyarrow]
 10  Bwd Packet Length Max        float[pyarrow] 
 11  Bwd Packet Length Min        float[pyarrow] 
 12  Bwd Packet Length Mean       double[pyarrow]
 13  Bwd Packet Length Std        double[pyarrow]
 14  Flow Bytes/s                 double[pyarrow]
 15  Flow Packets/s               double

In [14]:
ds[np.isinf(ds.select_dtypes(include=np.number)).any(axis=1)].Label.value_counts()

Label
Benign           1099
Infilteration      46
Name: count, dtype: int64[pyarrow]

In [15]:
ds[np.isinf(ds.select_dtypes(include=np.number)).any(axis=1)]

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1431849,6,0,2,0,31.0,0.0,31.0,0.0,15.5,21.920310,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,40,0,0.0,0.0,0.0,31.0,20.666667,17.897858,320.333333,0,1,0,0,1,0,0,0,0.0,31.0,15.5,0.0,0,0,0,0,0,0,2,31,0,0,980,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1433506,6,0,2,0,31.0,0.0,31.0,0.0,15.5,21.920310,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,40,0,0.0,0.0,0.0,31.0,20.666667,17.897858,320.333333,0,1,0,0,1,0,0,0,0.0,31.0,15.5,0.0,0,0,0,0,0,0,2,31,0,0,946,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1434527,6,0,2,0,31.0,0.0,31.0,0.0,15.5,21.920310,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,40,0,0.0,0.0,0.0,31.0,20.666667,17.897858,320.333333,0,1,0,0,1,0,0,0,0.0,31.0,15.5,0.0,0,0,0,0,0,0,2,31,0,0,294,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1434587,6,0,2,0,38.0,0.0,38.0,0.0,19.0,26.870058,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,40,0,0.0,0.0,0.0,38.0,25.333333,21.939310,481.333333,0,1,0,0,1,0,0,0,0.0,38.0,19.0,0.0,0,0,0,0,0,0,2,38,0,0,176,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1434831,6,0,2,0,53.0,0.0,53.0,0.0,26.5,37.476659,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,40,0,0.0,0.0,0.0,53.0,35.333333,30.599564,936.333333,0,1,0,0,1,0,0,0,0.0,53.0,26.5,0.0,0,0,0,0,0,0,2,53,0,0,245,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16161432,6,0,1,1,58.0,31.0,58.0,58.0,58.0,0.000000,31.0,31.0,31.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,20,20,0.0,0.0,31.0,58.0,49.000000,15.588457,243.000000,0,1,0,0,1,0,0,0,1.0,73.5,58.0,31.0,0,0,0,0,0,0,1,58,1,31,68,68,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infilteration
16177874,6,0,2,0,53.0,0.0,53.0,0.0,26.5,37.476659,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,40,0,0.0,0.0,0.0,53.0,35.333333,30.599564,936.333333,0,1,0,0,1,0,0,0,0.0,53.0,26.5,0.0,0,0,0,0,0,0,2,53,0,0,131,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infilteration
16181398,6,0,2,0,31.0,0.0,31.0,0.0,15.5,21.920310,0.0,0.0,0.0,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,40,0,0.0,0.0,0.0,31.0,20.666667,17.897858,320.333333,0,1,0,0,1,0,0,0,0.0,31.0,15.5,0.0,0,0,0,0,0,0,2,31,0,0,122,-1,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infilteration
16

In [16]:
ds = ds[~np.isinf(ds.select_dtypes(include=np.number)).any(axis=1)]

In [17]:
ds["Attack"] = ds["Label"].apply(lambda x: 1 if x != "Benign" else 0).astype("bool")

In [18]:
ds.describe()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,10820836.0,1.082084e+07,10820836.0,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,10820836.0,10820836.0,10820836.0,10820836.0,10820836.0,10820836.0,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07,1.082084e+07
mean,8.087906e+00,1.755072e+07,3.455673e+01,9.192370e+00,1.450181e+03,7.080574e+03,2.921326e+02,8.675383e+00,1.230545e+02,1.056497e+02,5.088795e+02,2.333959e+01,8.097099e+02,1.985700e+02,5.964704e+05,3.980312e+03,2.618376e+07,8.146706e+07,9.760533e+06,3.340714e+06,1.711490e+07,3.094394e+07,9.953723e+07,9.471133e+06,3.470634e+06,1.139521e+07,2.384618e+07,2.297287e+07,3.917800e+06,4.348709e+05,4.291387e-02,0.0,1.802079e-04,0.0,3.721678e+02,1.962204e+02,2.997746e+03,9.750870e+02,8.532274e+00,5.664930e+02,3.487025e+02,1.767857e+02,1.930324e+05,2.312113e-03,4.291387e-02,2.811150e-01,5.473757e-01,2.295595e-01,1.532303e-02,1.802079e-04,2.811151e-01,4.709304e-01,2.783823e+03,1.230545e+02,8.097099e+02,0.0,0.0,0.0,0.0,0.0,0.0,3.455673e+01,1.450181e+03,9.192370e+00,7.080540e+03,1.018636e+04,1.287343e+04,2.982517e+01,1.809680e+01,4.768384e+05,1.333987e+06,3.936085e+05,1.731976e+05,1.666809e+07,3.793741e+06,8.023772e+06,6.909935e+06
std,4.556638e+00,6.049764e+08,1.863004e+03,2.008376e+02,7.614071e+04,2.871466e+05,3.359054e+02,2.383230e+01,2.479685e+03,1.293054e+02,5.425371e+02,5.115383e+01,1.737733e+04,8.988252e+02,2.888198e+07,4.105655e+05,4.025511e+08,5.976214e+08,8.077489e+08,9.156854e+08,6.049741e+08,4.804407e+08,6.444778e+08,8.077476e+08,9.156888e+08,3.098330e+07,3.803559e+08,3.086984e+08,1.233357e+07,4.656384e+06,2.026630e-01,0.0,1.342294e-02,0.0,1.501915e+04,3.998100e+03,4.085038e+05,1.324471e+04,2.150273e+01,5.488170e+02,7.424123e+03,2.572121e+02,6.094536e+06,4.802882e-02,2.026630e-01,4.495435e-01,4.977505e-01,4.205496e-01,1.228342e-01,1.342294e-02,4.495435e-01,1.167465e+00,2.109845e+04,2.479685e+03,1.737733e+04,0.0,0.0,0.0,0.0,0.0,0.0,1.863004e+03,7.614071e+04,2.008376e+02,2.871340e+05,1.742797e+04,2.399703

In [19]:
ds.drop(columns=["Bwd PSH Flags","Bwd URG Flags","Fwd Avg Bytes/Bulk","Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate"], inplace=True)

In [20]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10820836 entries, 0 to 16232987
Data columns (total 71 columns):
 #   Column                       Dtype          
---  ------                       -----          
 0   Protocol                     uint8[pyarrow] 
 1   Flow Duration                int64[pyarrow] 
 2   Total Fwd Packets            uint32[pyarrow]
 3   Total Backward Packets       uint32[pyarrow]
 4   Total Length of Fwd Packets  double[pyarrow]
 5   Total Length of Bwd Packets  double[pyarrow]
 6   Fwd Packet Length Max        float[pyarrow] 
 7   Fwd Packet Length Min        float[pyarrow] 
 8   Fwd Packet Length Mean       double[pyarrow]
 9   Fwd Packet Length Std        double[pyarrow]
 10  Bwd Packet Length Max        float[pyarrow] 
 11  Bwd Packet Length Min        float[pyarrow] 
 12  Bwd Packet Length Mean       double[pyarrow]
 13  Bwd Packet Length Std        double[pyarrow]
 14  Flow Bytes/s                 double[pyarrow]
 15  Flow Packets/s               double

In [21]:
compute_ratio(ds,'Label')

                          Ratio
Label                          
Benign                    87.75
DDoS attacks-LOIC-HTTP     5.32
DDOS attack-HOIC           1.84
DoS attacks-Hulk           1.34
Bot                        1.34
Infilteration              1.05
SSH-Bruteforce             0.87
DoS attacks-GoldenEye      0.38
DoS attacks-Slowloris      0.09
DDOS attack-LOIC-UDP       0.02
Brute Force -Web           0.01
Brute Force -XSS           0.00
SQL Injection              0.00
DoS attacks-SlowHTTPTest   0.00
FTP-BruteForce             0.00


In [ ]:
ds.Label = ds.Label.replace('Infilteration', 'Infiltration')
ds.Label = ds.Label.replace('DoS attacks-Hulk', 'DoS Hulk')
ds.Label = ds.Label.replace('SSH-Bruteforce ', 'SSH-Patator')
ds.Label = ds.Label.replace('FTP-BruteForce', 'FTP-Patator')
ds.Label = ds.Label.replace('Brute Force -Web', 'Web Attack  Brute Force')
ds.Label = ds.Label.replace('Brute Force -XSS', 'Web Attack  XSS')
ds.Label = ds.Label.replace('DDOS attack-HOIC', 'Web Attack  XSS')

In [20]:
ds.to_parquet("../datasets/CICIDS-2018.parquet", index=False)